In [1]:
import google_auth_oauthlib.flow
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import pandas as pd
import datetime
import numpy as np

Авторизуемся с помощью клиентского токена

In [2]:
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']

CLIENT_FILE = 'client_id.json'

state = {}
flow = google_auth_oauthlib.flow.Flow.from_client_secrets_file(
    CLIENT_FILE,
    scopes=SCOPES,
    state=state)

flow.redirect_uri = "urn:ietf:wg:oauth:2.0:oob"
# Ссылка по которой можно авторизовать 
print("\n\n".join(flow.authorization_url()))

https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=488672165176-7121s50pdtmsd1sria9p62d5gegj7e1f.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fanalytics.readonly&state=4pSRrbHUm94MqtxWRNbDStJK7Eac0l&access_type=offline

4pSRrbHUm94MqtxWRNbDStJK7Eac0l


In [3]:
token = flow.fetch_token(code='4/IwG5KSfuikSu389gsg2VTLHlwsz4vgr3xfMRI7-KxrbmxK5cetFh0AM')
credentials=flow.credentials

Либо авторизуемся с помощью сервисного аккаунта

In [4]:
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']

SERVICE_ACCOUNT_FILE = 'local-grove-232309-6fbe879cfe7d.json'

credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

FileNotFoundError: [Errno 2] No such file or directory: 'local-grove-232309-6fbe879cfe7d.json'

Создаем сервис для запросов к Google Analytics Core Reporting v4

In [4]:
gam = build('analytics', 'v4', credentials=credentials)

Получаем clientId из custom dimension. Добавляем показатели новые пользователи (ga:newUsers) и посещения (ga:visits), чтобы потом оставить только новых посетителей. Забираем все данные с помощью цикла

In [5]:
def createPayloadForCoreReportingAPI(viewId,metrics,dimensions,date_ranges,dimension_filter_clauses,next_page_token=""):
    metrics_list = [{'expression':metric} for metric in metrics]
    dimensions_list = [{'name':dimension} for dimension in dimensions]
    date_ranges_list = [{'startDate':date1,'endDate':date2} for date1,date2 in date_ranges]
    payload_body = {
        "reportRequests": [
            {
                "viewId": viewId,
                "dateRanges": date_ranges_list,
                "metrics": metrics_list,
                "dimensions": dimensions_list,
                "pageSize":"10000",
                "samplingLevel":  "LARGE",
                "dimensionFilterClauses": dimension_filter_clauses,
                "pageToken":next_page_token
            }]
      }
    return payload_body

def getCoreReportingAPIReport(gam,viewId,metrics,dimensions,date_ranges,dimension_filter_clauses):
    payload_body = createPayloadForCoreReportingAPI(viewId,metrics,dimensions,date_ranges,dimension_filter_clauses)
    data = gam.reports().batchGet(body=payload_body).execute()
    print (len(data['reports'][0]['data']['rows']))
    next_page_token = data['reports'][0].get('nextPageToken')
    while next_page_token:
        payload_body = createPayloadForCoreReportingAPI(viewId,metrics,dimensions,date_ranges,dimension_filter_clauses,next_page_token=next_page_token)
        next_data = gam.reports().batchGet(body=payload_body).execute()
        print (len(next_data['reports'][0]['data']['rows']))
        next_page_token = next_data['reports'][0].get('nextPageToken')
        data['reports'][0]['data']['rows'] = data['reports'][0]['data']['rows'] + next_data['reports'][0]['data']['rows']
    return data

def createListOfDictsFromGAData(ga_data):
    dimensions_header = ga_data['reports'][0]['columnHeader']['dimensions']
    metrics_header = ga_data['reports'][0]['columnHeader']['metricHeader']['metricHeaderEntries']
    all_data = []
    for row in ga_data['reports'][0]['data']['rows']:
        d = {}
        for i,dimension_value in enumerate(row['dimensions']):
            d[dimensions_header[i]] = dimension_value
        for i,metric_value in enumerate(row['metrics'][0]['values']):
            d[metrics_header[i]['name']] = metric_value
        all_data.append(d)
    return all_data

# Параметры запроса

viewId = '170933264' # ID профиля
date_ranges = [('2019-03-15','2019-03-22')] # Диапазоны сбора данных (поддерживается только один диапазон)
metrics = ['ga:newUsers','ga:visits'] # Метрики
dimensions = ['ga:dimension10'] # Измерения
dimension_filter_clauses = [ # Фильтр (если не нужен, то можно оставить пустой список)
    {
        "filters": [
            {
              "dimensionName": "ga:eventAction",
              "operator": "EXACT",
              "expressions": ["совершил подробный поиск объявлений"]
            }
        ]
    }
]

ga_data = getCoreReportingAPIReport(gam,viewId,metrics,dimensions,date_ranges,dimension_filter_clauses)
report_df = pd.DataFrame(createListOfDictsFromGAData(ga_data)) # Датафрейм с данными
print (len(report_df))


4164
4164


Обрабатываем датафрейм report_df (приводим строки с числами к целочисленному формату)

In [6]:
report_df['ga:newUsers'] = report_df['ga:newUsers'].astype(int)
report_df['ga:visits'] = report_df['ga:visits'].astype(int)

client_ids = report_df[report_df['ga:newUsers'] > 0].sort_values('ga:visits',ascending=False).head(100)['ga:dimension10']


In [7]:
report_df[(report_df['ga:newUsers'] > 0) & (report_df['ga:visits'] > 1)]

,ga:dimension10,ga:newUsers,ga:visits
3,1001866101.1516638882,1,2
10,1004765897.1552728303,1,2
25,1011268470.1552835339,1,2
61,1031793442.1553002015,1,2
62,1032516467.1552828428,1,2
65,1033626993.1552829544,1,3
71,1035059619.1524197791,1,2
97,1046549852.1532797791,1,2
105,1050922945.1552812140,1,2
116,1055764884.1524915590,1,2


In [8]:
user_info = gam.userActivity().search(body={
        "viewId": "170933264",
        "user": {
            "type": "CLIENT_ID",
            "userId": "106810630.1552814152"
        },
        "dateRange": {
            "startDate": "2019-03-15",
            "endDate": "2019-03-22",
        }
    }).execute()

In [9]:
user_info['sessions']

[{'sessionId': '1553066684',
  'deviceCategory': 'mobile',
  'platform': 'Android',
  'dataSource': 'web',
  'activities': [{'activityTime': '2019-03-20T14:28:32.003469Z',
    'source': 'google',
    'medium': 'organic',
    'channelGrouping': 'Organic Search',
    'campaign': '(not set)',
    'keyword': '(not provided)',
    'hostname': 'krasnoyarsk.n1.ru',
    'landingPagePath': '/kupit/kvartiry/vtorichka/',
    'activityType': 'PAGEVIEW',
    'customDimension': [{'index': 1, 'value': 'n1-mobile'},
     {'index': 2, 'value': 'Красноярский край'},
     {'index': 3, 'value': 'search'},
     {'index': 4, 'value': 'offerCard'},
     {'index': 5},
     {'index': 6, 'value': 'living:flats'},
     {'index': 7, 'value': 'sell'},
     {'index': 8},
     {'index': 9},
     {'index': 10, 'value': '106810630.1552814152'},
     {'index': 11},
     {'index': 12},
     {'index': 13},
     {'index': 14, 'value': 'rooms,price,mDistrict,district'},
     {'index': 15, 'value': '90-144'},
     {'index':

In [10]:
print (datetime.datetime.now())

activities_data = []

cnt = len(client_ids)
for i,client_id in enumerate(client_ids):
    # print ('{0} in {1}'.format(i+1,cnt))
    try:
        user_info = gam.userActivity().search(body={
        "viewId": "170933264",
        "user": {
            "type": "CLIENT_ID",
            "userId": client_id
        },
        "dateRange": {
            "startDate": "2019-03-15",
            "endDate": "2019-03-22",
        }
    }).execute()
    except HttpError:
        print ("{0}: HttpError".format(client_id))
        

    
    for session in user_info.get('sessions'):
        session_keys = session.keys()
        for activity in session['activities']:
            d = activity
            for key in session_keys:
                if key != 'activities':
                    d[key] = session[key]
            if activity['activityType'] == 'EVENT':
                for key in activity['event'].keys():
                    d[key] = activity['event'][key]
            d['clientId'] = client_id
            activities_data.append(d)
    
print (datetime.datetime.now())

2019-04-05 15:33:19.182135
2019-04-05 15:34:22.259425


In [11]:
print(len(activities_data))

5952


In [12]:
activity_df = pd.DataFrame(activities_data)
activity_df['activityTime'] = pd.to_datetime(activity_df['activityTime'])
activity_df.columns

Index(['activityTime', 'activityType', 'campaign', 'channelGrouping',
       'clientId', 'customDimension', 'dataSource', 'deviceCategory', 'event',
       'eventAction', 'eventCategory', 'eventCount', 'eventLabel', 'hostname',
       'keyword', 'landingPagePath', 'medium', 'pageview', 'platform',
       'sessionDate', 'sessionId', 'source'],
      dtype='object')

In [13]:
activity_df[activity_df.eventAction == 'совершил подробный поиск объявлений'].groupby(['clientId'],as_index=False).agg({'activityTime':'min'})

,clientId,activityTime
0,106810630.1552814152,2019-03-17 16:17:35.174665
1,1102002913.1552844851,2019-03-18 00:48:21.947322
2,115531401.1552665714,2019-03-15 23:02:43.667293
3,1180531388.1552737240,2019-03-16 18:54:57.379962
4,1181048176.1553243510,2019-03-22 15:32:19.522014
5,1184398747.1552812369,2019-03-17 15:51:34.415361
6,1190322833.1553033613,2019-03-20 01:13:59.560593
7,1198228301.1553162099,2019-03-21 16:57:36.787250
8,1213691535.1552921951,2019-03-18 18:10:17.893445
9,1266869650.1552807217,2019-03-17 14:20:54.574299


In [14]:
activity_df.groupby(['clientId'],as_index=False).agg({'activityTime':'min'})

,clientId,activityTime
0,106810630.1552814152,2019-03-17 16:15:55.565066
1,1102002913.1552844851,2019-03-18 00:47:33.609295
2,115531401.1552665714,2019-03-15 23:01:54.167273
3,1180531388.1552737240,2019-03-16 18:54:04.841519
4,1181048176.1553243510,2019-03-22 15:31:54.885445
5,1184398747.1552812369,2019-03-17 15:46:11.379580
6,1190322833.1553033613,2019-03-20 01:13:18.660866
7,1198228301.1553162099,2019-03-21 16:55:09.154488
8,1213691535.1552921951,2019-03-18 18:09:09.331096
9,1266869650.1552807217,2019-03-17 14:20:17.782236


In [16]:
activity_df

,activityTime,activityType,campaign,channelGrouping,clientId,customDimension,dataSource,deviceCategory,event,eventAction,...,eventLabel,hostname,keyword,landingPagePath,medium,pageview,platform,sessionDate,sessionId,source
0,2019-03-22 19:28:09.289190,PAGEVIEW,(not set),Organic Search,737874267.1552921540,"[{'index': 1, 'value': 'n1-mobile'}, {'index':...",web,mobile,NaN,NaN,...,NaN,krasnoyarsk.n1.ru,(not provided),"/search/?is_newbuilding=false&rooms=3,4&addres...",organic,"{'pagePath': '/view/28809415/', 'pageTitle': '...",Android,2019-03-22,1553257615,google
1,2019-03-22 19:26:55.470664,PAGEVIEW,(not set),Organic Search,737874267.1552921540,"[{'index': 1, 'value': 'n1-mobile'}, {'index':...",web,mobile,NaN,NaN,...,NaN,krasnoyarsk.n1.ru,(not provided),"/search/?is_newbuilding=false&rooms=3,4&addres...",organic,{'pagePath': '/search/?is_newbuilding=false&ro...,Android,2019-03-22,1553257615,google
2,2019-03-22 19:26:55.464682,EVENT,Krasnoyarsk_24_new|396651841,Paid Search,737874267.1552921540,"[{'index': 1, 'value': 'n1-mobile'}, {'index':...",web,mobile,"{'eventCategory': 'пользователь', 'eventAction...",совершил подробный поиск объявлений,...,(not set),krasnoyarsk.n1.ru,n1 красноярск,/kupit/kvartiry/vtorichka/,cpc,NaN,Android,2019-03-22,1553257590,google
3,2019-03-22 19:26:30.315114,PAGEVIEW,Krasnoyarsk_24_new|396651841,Paid Search,737874267.1552921540,"[{'index': 1, 'value': 'n1-mobile'}, {'index':...",web,mobile,NaN,NaN,...,NaN,krasnoyarsk.n1.ru,n1 красноярск,/kupit/kvartiry/vtorichka/,cpc,"{'pagePath': '/kupit/kvartiry/vtorichka/', 'pa...",Android,2019-03-22,1553257590,google
4,2019-03-22 14:54:11.478100,PAGEVIEW,Krasnoyarsk_24_new|396651841,Paid Search,737874267.1552921540,"[{'index': 1, 'value': 'n1-mobile'}, {'index':...",web,mobile,NaN,NaN,...,NaN,krasnoyarsk.n1.ru,n1 красноярск,/kupit/kvartiry/vtorichka/,cpc,"{'pagePath': '/kupit/kvartiry/vtorichka/', 'pa...",Android,2019-03-22,1553241251,google
5,2019-03-22 14:53:53.425786,PAGEVIEW,(not set),Organic Search,737874267.1552921540,"[{'index': 1, 'value': 'n1-mobile'}, {'index':...",web,mobile,NaN,NaN,...,NaN,krasnoyarsk.n1.ru,(not provided),/view/29577653/,organic,{'pagePath': '/search/?is_newbuilding=false&ro...,Android,2019-03-22,1553241229,google
6,2019-03-22 14:53:49.839259,PAGEVIEW,(not set),Organic Search,737874267.1552921540,"[{'index': 1, 'value': 'n1-mobile'}, {'index':...",web,mobile,NaN,NaN,...,NaN,krasnoyarsk.n1.ru,(not provided),/view/29577653/,organic,"{'pagePath': '/view/29577653/', 'pageTitle': '...",Android,2019-03-22,1553241229,google
7,2019-03-22 14:53:30.914326,EVENT,Krasnoyarsk_24_new|396651841,Paid Search,737874267.1552921540,"[{'index': 1, 'value': 'n1-mobile'}, {'index':...",web,mobile,"{'eventCategory': 'пользователь', 'eventAction...",открыл полноэкранную галерею объявления,...,(not set),krasnoyarsk.n1.ru,n1 красноярск,(not set),cpc,NaN,Android,2019-03-22,1553241210,google
8,2019-03-22 14:53:28.952386,PAGEVIEW,(not set),Organic Search,737874267.1552921540,"[{'index': 1, 'value': 'n1-mobile'}, {'index':...",web,mobile,NaN,NaN,...,NaN,krasnoyarsk.n1.ru,(not provided),/view/29768326/,organic,"{'pagePath': '/view/29768326/', 'pageTitle': '...",Android,2019-03-22,1553241208,google
9,2019-03-22 14:39:09.503593,EVENT,Krasnoyarsk_24_new|396651841,Paid Search,737874267.1552921540,"[{'index': 1, 'value': 'n1-mobile'}, {'index':...",web,mobile,"{'eventCategory': 'пользователь', 'eventAction...",открыл полноэкранную галерею объявления,...,(not set),krasnoyarsk.n1.ru,n1 красноярск,(not set),cpc,NaN,Android,2019-03-22,1553240349,google


In [17]:
clients_event_stats = activity_df.groupby(['clientId'],as_index=False).agg({'activityTime':'min'}).\
merge(
    activity_df[activity_df.eventAction == 'совершил подробный поиск объявлений'].groupby(['clientId'],as_index=False).agg({'activityTime':'min'}).\
    rename(columns={'activityTime':'eventTime'})
    ,on='clientId',how='left').\
merge(activity_df.groupby(['clientId'],as_index=False).agg({'sessionId':'nunique'}),
      on='clientId',how='left')


In [18]:
clients_event_stats

,clientId,activityTime,eventTime,sessionId
0,106810630.1552814152,2019-03-17 16:15:55.565066,2019-03-17 16:17:35.174665,11
1,1102002913.1552844851,2019-03-18 00:47:33.609295,2019-03-18 00:48:21.947322,12
2,115531401.1552665714,2019-03-15 23:01:54.167273,2019-03-15 23:02:43.667293,10
3,1180531388.1552737240,2019-03-16 18:54:04.841519,2019-03-16 18:54:57.379962,14
4,1181048176.1553243510,2019-03-22 15:31:54.885445,2019-03-22 15:32:19.522014,14
5,1184398747.1552812369,2019-03-17 15:46:11.379580,2019-03-17 15:51:34.415361,28
6,1190322833.1553033613,2019-03-20 01:13:18.660866,2019-03-20 01:13:59.560593,18
7,1198228301.1553162099,2019-03-21 16:55:09.154488,2019-03-21 16:57:36.787250,61
8,1213691535.1552921951,2019-03-18 18:09:09.331096,2019-03-18 18:10:17.893445,5
9,1266869650.1552807217,2019-03-17 14:20:17.782236,2019-03-17 14:20:54.574299,18


In [19]:
clients_event_stats['eventDelta'] = (clients_event_stats['eventTime'] - clients_event_stats['activityTime'])

In [20]:
clients_event_stats

,clientId,activityTime,eventTime,sessionId,eventDelta
0,106810630.1552814152,2019-03-17 16:15:55.565066,2019-03-17 16:17:35.174665,11,00:01:39.609599
1,1102002913.1552844851,2019-03-18 00:47:33.609295,2019-03-18 00:48:21.947322,12,00:00:48.338027
2,115531401.1552665714,2019-03-15 23:01:54.167273,2019-03-15 23:02:43.667293,10,00:00:49.500020
3,1180531388.1552737240,2019-03-16 18:54:04.841519,2019-03-16 18:54:57.379962,14,00:00:52.538443
4,1181048176.1553243510,2019-03-22 15:31:54.885445,2019-03-22 15:32:19.522014,14,00:00:24.636569
5,1184398747.1552812369,2019-03-17 15:46:11.379580,2019-03-17 15:51:34.415361,28,00:05:23.035781
6,1190322833.1553033613,2019-03-20 01:13:18.660866,2019-03-20 01:13:59.560593,18,00:00:40.899727
7,1198228301.1553162099,2019-03-21 16:55:09.154488,2019-03-21 16:57:36.787250,61,00:02:27.632762
8,1213691535.1552921951,2019-03-18 18:09:09.331096,2019-03-18 18:10:17.893445,5,00:01:08.562349
9,1266869650.1552807217,2019-03-17 14:20:17.782236,2019-03-17 14:20:54.574299,18,00:00:36.792063


In [21]:
clients_event_stats['eventDelta'] = (clients_event_stats['eventTime'] - clients_event_stats['activityTime'])/ np.timedelta64(1, 's')
print (clients_event_stats['eventDelta'].describe())


count     100.000000
mean      114.836936
std       176.850241
min         3.886428
25%        46.880852
50%        62.719038
75%       114.251827
max      1107.427420
Name: eventDelta, dtype: float64


In [22]:
bins = [0,30,60,120,240,1200]
clients_event_stats['eventDeltaBin'] = pd.cut(clients_event_stats['eventDelta'], bins, right=False, labels=None, retbins=False, precision=3, include_lowest=False)
clients_event_stats.groupby(['eventDeltaBin']).agg({'clientId':'nunique'})


,clientId
eventDeltaBin,
"[0, 30)",14
"[30, 60)",34
"[60, 120)",30
"[120, 240)",14
"[240, 1200)",8


In [23]:
activity_df[(activity_df.eventAction != 'совершил подробный поиск объявлений') 
            & (activity_df.activityType == 'EVENT')].groupby(['eventCategory','eventAction','source']).agg({'activityTime':'count','sessionId':'nunique'})


activityTime  \
eventCategory eventAction                                        source                         
searchform    show                                               mobilen1                   1   
пользователь  выбрал адрес в форме поиска                        mobilen1                   1   
              выбрал цену в форме поиска                         mobilen1                   1   
              добавил в избранное объявление в карточке объяв... (direct)                   3   
                                                                 away.vk.com                7   
                                                                 google                    10   
                                                                 yandex                     4   
              добавил в избранное объявление на странице поис... (direct)                   2   
                                                                 away.vk.com                1   
                                                                 google                    19   
                                                                 yandex                     6   
              нажал на карте в форме поиска                      mobilen1                   1   
              нажал найти в форме поиска                         mobilen1                   1   
              нажал поделиться объявлением в социальной сети ... (direct)                   4   
                                                                 away.vk.com               13   
                                                                 google                     3   
                                                                 yandex                     1   
              нажал позвонить по контактам объявления в карто... google                     9   
                                                                 yabs.yandex.ru             4   
                                                                 yandex                     7   
              открыл карточку избранного на странице избранного  (direct)                   1   
                                                                 away.vk.com                5   
                                                                 google                    62   
                                                                 yandex                    10   
              открыл контакты объявления в карточке объявления   google                    60   
                                                                 yabs.yandex.ru             5   
                                                                 yandex                    27   
                                                                 yandex.ru                  1   
              открыл контакты объявления на странице поиска о... yandex                     1   
              открыл полноэкранную галерею объявления            (direct)                   1   
                                                                 google                   218   
                                                                 yabs.yandex.ru            13   
                                                                 yandex                   165   
                                                                 yandex.ru                 16   
              перешел на карту с баннера на странице поиска о... mobilen1                   1   
              увидел блок рекомендаций в карточке объявления     yabs.yandex.ru             3   
                                                                 yandex                     5   
              увидел страницу выдачи менее 6 объявлений в пои... mobilen1                   3   
              удалил из избранного в карточке объявления         away.vk.com                1   
                                                                 google                     1   
       

In [24]:
activity_df = activity_df.merge(clients_event_stats[['clientId','eventTime']],on='clientId',how='left')

In [25]:
activity_df[(activity_df.eventAction != 'совершил подробный поиск объявлений')
            & (activity_df.activityType == 'EVENT') 
            & (activity_df.activityTime > activity_df.eventTime)].groupby(['eventCategory','eventAction']).agg({'activityTime':'count','sessionId':'nunique','clientId':'nunique'})


activityTime  \
eventCategory eventAction                                                        
searchform    show                                                           1   
пользователь  выбрал адрес в форме поиска                                    1   
              выбрал цену в форме поиска                                     1   
              добавил в избранное объявление в карточке объяв...            24   
              добавил в избранное объявление на странице поис...            28   
              нажал на карте в форме поиска                                  1   
              нажал найти в форме поиска                                     1   
              нажал поделиться объявлением в социальной сети ...            21   
              нажал позвонить по контактам объявления в карто...            20   
              открыл карточку избранного на странице избранного             78   
              открыл контакты объявления в карточке объявления              93   
              открыл контакты объявления на странице поиска о...             1   
              открыл полноэкранную галерею объявления                      413   
              перешел на карту с баннера на странице поиска о...             1   
              увидел блок рекомендаций в карточке объявления                 8   
              увидел страницу выдачи менее 6 объявлений в пои...             3   
              удалил из избранного в карточке объявления                     2   
              удалил из избранного на странице избранного                   26   

                                                                  sessionId  \
eventCategory eventAction                                                     
searchform    show                                                        1   
пользователь  выбрал адрес в форме поиска                                 1   
              выбрал цену в форме поиска                                  1   
              добавил в избранное объявление в карточке объяв...         14   
              добавил в избранное объявление на странице поис...         15   
              нажал на карте в форме поиска                               1   
              нажал найти в форме поиска                                  1   
              нажал поделиться объявлением в социальной сети ...          7   
              нажал позвонить по контактам объявления в карто...         18   
              открыл карточку избранного на странице избранного          23   
              открыл контакты объявления в карточке объявления           62   
              открыл контакты объявления на странице поиска о...          1   
              открыл полноэкранную галерею объявления                   218   
              перешел на карту с баннера на странице поиска о...          1   
              увидел блок рекомендаций в карточке объявления              4   
              увидел страницу выдачи менее 6 объявлений в пои...          1   
              удалил из избранного в карточке объявления                  2   
              удалил из избранного на странице избранного                 5   

                                                                  clientId  
eventCategory eventAction                                                   
searchform    show                                                       1  
пользователь  выбрал адрес в форме поиска                                1  
              выбрал цену в форме поиска                                 1  
              добавил в избранное объявление в карточке объяв...         8  
              добавил в избранное объявление на странице поис...        10  
              нажал на карте в форме поиска                              1  
              нажал найти в форме поиска                                 1  
              нажал поделиться объявлением в социальной сети ...         3  
              нажал позвонить по контактам объявления в карто...        14

In [26]:
clients_stats_2 = activity_df[(activity_df.eventAction != 'совершил подробный поиск объявлений') & (activity_df.activityType == 'EVENT') & (activity_df.activityTime > activity_df.eventTime) & (activity_df.eventAction == 'открыл контакты объявления в карточке объявления')].sort_values('activityTime').drop_duplicates(['clientId'])[['clientId','activityTime','eventTime']]
clients_stats_2['eventDelta'] = (clients_stats_2['activityTime'] - clients_stats_2['eventTime'])/ np.timedelta64(1, 'h')
clients_stats_2['eventDelta'].describe()


count     27.000000
mean      20.535998
std       30.860319
min        0.028983
25%        0.145970
50%        4.535356
75%       26.950579
max      121.385659
Name: eventDelta, dtype: float64

In [27]:
bins = [0,1,6,12,24,122]
clients_stats_2['eventDeltaBin'] = pd.cut(clients_stats_2['eventDelta'], bins, right=False, labels=None, retbins=False, precision=3, include_lowest=False)
clients_stats_2.groupby(['eventDeltaBin']).agg({'clientId':'nunique'})


,clientId
eventDeltaBin,
"[0, 1)",11
"[1, 6)",3
"[6, 12)",2
"[12, 24)",3
"[24, 122)",8
